In [ ]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# ================== 配置 ==================
load_dotenv()
api_base = os.getenv("OPENAI_BASE_URL")
api_key = os.getenv("OPENAI_API_KEY")

print("=" * 60)
print("LangChain 最新RAG系统 - 完整版")
print("=" * 60)

# ================== 步骤1：加载文本文件 ==================
print("\n[1/5] 加载文本文件...")
loader = TextLoader("./test/file/sample.txt", encoding="utf-8")
docs = loader.load()
print(f"     ✓ 加载 {len(docs)} 个文档")

# ================== 步骤2：分割文本 ==================
print("[2/5] 分割文本...")
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
)
chunks = text_splitter.split_documents(docs)
print(f"     ✓ 分割成 {len(chunks)} 个块")

# ================== 步骤3：创建向量存储 ==================
print("[3/5] 创建向量存储...")
embeddings = OpenAIEmbeddings(
    model="baai/bge-m3",
    api_key=api_key,
    base_url=api_base,
)
vector_store = FAISS.from_documents(chunks, embeddings)
retriever = vector_store.as_retriever(search_kwargs={"k": 3})
print("     ✓ 向量存储已创建")

# ================== 步骤4：创建LLM ==================
print("[4/5] 初始化语言模型...")
llm = ChatOpenAI(
    model="gpt-4o-mini",
    api_key=api_key,
    base_url=api_base,
    temperature=0.7,
)
print("     ✓ LLM已初始化")

# ================== 步骤5：构建RAG链（使用最新的Runnable API）==================
print("[5/5] 构建RAG链...")

# 定义提示词模板
prompt = ChatPromptTemplate.from_template("""
根据以下检索到的上下文回答问题。

<context>
{context}
</context>

<question>
{question}
</question>

请提供详细且准确的回答：""")

# ✅ 这是最新的构建RAG链的方式（替代了过时的RetrievalQA.from_chain_type）
rag_chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
)
print("     ✓ RAG链已构建完成")

# ================== 测试系统 ==================
print("\n" + "=" * 60)
print("系统测试")
print("=" * 60)

# 单个问题测试
question = "LangChain有哪些主要特性？"
print(f"\n🔍 问题: {question}")
print("-" * 60)

result = rag_chain.invoke(question)
print(f"📝 回答: {result.content}")

# ================== 流式输出（可选）==================
print("\n" + "=" * 60)
print("流式输出测试")
print("=" * 60)

question2 = "什么是文本分割？"
print(f"\n🔍 问题: {question2}")
print("-" * 60)
print("📝 回答: ", end="", flush=True)

# 使用stream方法获取流式输出
for chunk in rag_chain.stream(question2):
    print(chunk.content, end="", flush=True)
print()

print("\n" + "=" * 60)
print("✓ 系统运行完毕！")
print("=" * 60)